# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Bidding

In [2]:
vuln_ns, vuln_ew = False, True
hand = 'A.8xx.Jxx.AQxxxx'

auction = ["1S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?


[{'call': '2C',
  'insta_score': 0.621,
  'expected_score': -131,
  'adjustment': 31},
 {'call': 'PASS',
  'insta_score': 0.375,
  'expected_score': -133,
  'adjustment': 19}]

In [3]:
auction = ["1S","PASS","1N","PASS","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']


[{'call': 'X', 'insta_score': 0.892, 'expected_score': 3, 'adjustment': 45},
 {'call': 'PASS', 'insta_score': 0.107, 'expected_score': 15, 'adjustment': 5}]

In [4]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


Kxxxx.A.KTx.KJ9x A.8xx.Jxx.AQxxxx xx.QJTxx.A9xx.8x QJT98.K9xx.Q8x.T 0.99870 
 1S-P-1N-P-2C-X-2S-P-P-P (6.7)  
 1S-P-1N-P-2C-P-2S-P-P-P (6.7) 
Qxxxx.ATx.A.KJT8 A.8xx.Jxx.AQxxxx K8.KJ9x.QT98.9xx JT9xx.Qxx.Kxxxx. 0.99868 
 1S-P-1N-P-2C-X-P-2D-P-P-P (6.6)  
 1S-P-1N-P-2C-P-2S-P-P-P (7.6) 
KQ8xx.AJ.8x.KJTx A.8xx.Jxx.AQxxxx 9x.Kxxxx.A9x.98x JTxxx.QT9.KQTxx. 0.99861 
 1S-P-1N-P-2C-X-P-2D-P-P-P (8.5)  
 1S-P-1N-P-2C-P-2S-P-P-P (6.5) 
KJ9xx.AQx.K.JT8x A.8xx.Jxx.AQxxxx 8x.T9xx.AQxx.K9x QTxxx.KJx.T98xx. 0.99851 
 1S-P-1N-P-2C-X-P-2D-P-P-P (6.5)  
 1S-P-1N-P-2C-P-2S-P-P-P (7.8) 
K98xx.Jx.AQ.KJ8x A.8xx.Jxx.AQxxxx x.KQT9x.KT9xx.T9 QJTxxx.Axx.8xx.x 0.99846 
 1S-P-1N-P-2C-X-P-P-P (6.4)  
 1S-P-1N-P-2C-P-P-P (6.4) 
QJ98x.AK.Kx.JT9x A.8xx.Jxx.AQxxxx Tx.Txxx.AQTxx.Kx Kxxxx.QJ9x.98x.8 0.99834 
 1S-P-1N-P-2C-X-2S-P-P-P (7.9)  
 1S-P-1N-P-2C-P-2S-P-P-P (7.9) 
KQJTx.A.KTx.J8xx A.8xx.Jxx.AQxxxx xx.KQxx.Q8xx.KT9 98xxx.JT9xx.A9x. 0.99830 
 1S-P-1N-P-2C-X-2S-P-P-P (7.5)  
 1S-P-1N-P-2C-P-2S-P-P-P (7.5) 
KJTxx.K9

West always has the same cards because it's your hand and it's known to you

In [5]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction,2)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

98x.9xxxx.Qxx.T9 KQJx.AKJT.8x.KJ8 A.8xx.Jxx.AQxxxx Txxxx.Q.AKT9x.xx 0.63
9xx.AQTx.T9xxx.8 KQTx.Jx.AQx.KT9x A.8xx.Jxx.AQxxxx J8xxx.K9xx.K8.Jx 0.99
QJT8xx.Tx.9x.JT9 Kx.KQxx.A8xx.Kxx A.8xx.Jxx.AQxxxx 9xxx.AJ9x.KQTx.8 0.62
JT8xxx.T9x.KT.Tx Qxx.AQx.Axxx.K8x A.8xx.Jxx.AQxxxx K9x.KJxx.Q98x.J9 0.97
8xxxx.xx.AQx.J8x KQx.AKTxx.9xx.K9 A.8xx.Jxx.AQxxxx JT9x.QJ9.KT8x.Tx 0.27
J9xx.QJ9x.Tx.T9x KQxx.Kx.K8xx.KJx A.8xx.Jxx.AQxxxx T8xx.ATxx.AQ9x.8 0.98
QTxxx.Ax.Q8xxx.8 K9x.KT.AKT9.KT9x A.8xx.Jxx.AQxxxx J8xx.QJ9xxx.x.Jx 1.00
Txxx.9xx.A8xx.T8 QJ9x.AKx.Qx.KJ9x A.8xx.Jxx.AQxxxx K8xx.QJTx.KT9x.x 1.00
8xxx.QJxx.9xx.8x K9xx.KT.AQx.KJT9 A.8xx.Jxx.AQxxxx QJTx.A9xx.KT8x.x 1.00
8xx.Txx.9xx.T9xx JTxx.AKJ9.A8x.KJ A.8xx.Jxx.AQxxxx KQ9xx.Qxx.KQTx.8 1.00
